In [1]:
import numpy as np
import h5py
import os 
import json
import scipy.io as spio
import matplotlib.pyplot as plt 
import pickle
from matplotlib.mlab import PCA
from scipy.stats.distributions import chi2


# This is a python implementation of 

# Jin et al. (2015), Adaptive reference updating for vibration-based
# structural health monitoring under varying environmental conditions,
# Computers & Structures.

In [14]:
def Standardization_Data(X0,mu0,sds0,numobs,SD_type):
    import numpy as np 
    
    if SD_type == 'Mean_centered': return (X0 - np.matlib.repmat(mu0,numobs,1));
    elif SD_type == 'Z-score': return np.divide((X0 - np.matlib.repmat(mu0,numobs,1)),  np.matlib.repmat(sds0,numobs,1)) 
    
def hotelling_tsquared(pc):
    """`pc` should be the object returned by matplotlib.mlab.PCA()."""
    x = pc.a.T
    cov = pc.Wt.T.dot(np.diag(pc.s)).dot(pc.Wt) / (x.shape[1] - 1)
    w = np.linalg.solve(cov, x)
    t2 = (x * w).sum(axis=0)
    return t2

In [2]:
############################################################################
################## STEP2: RUN DAMAGE DETECTION USING APCA ##################
############################################################################
# 
# THIS CODE IS DEVELOPED BASED ON MY WORK IN SHM (2018)
#
# Ref.) 
# Jin et al. (2018),Vibration-based damage detection using 
# online learning algorithm for output-only structural health monitoring.
# Structural Health Monitoring.
#
# Originally coded by SS (2019.08.11) in MATLAB 2018b
# Modified BY SJ CHO (2019.10.15) in MATLAB 2019a
# Converted into Python BY BH Kim (2020.01.23) in Python 3.7.4

In [3]:
## Intialization and Add path required m-files-

####################### OPTIONS #######################

PCA_par={};

# Filename for monitoring data

file_format = '.h5' # or '.h5'

# PCA_par['fn_DB'] = 'DB_case0_lin_1e-05;
# PCA_par['fn_DB'] = 'DB_case1_lin_1e-05;
# PCA_par['fn_DB'] = 'DB_case0_bi_lin_1e-05'
# PCA_par['fn_DB'] = 'DB_case1_bi_lin_1e-05'

# PCA_par['fn_DB'] = 'DB_case0_lin_0.001';
# PCA_par['fn_DB'] = 'DB_case1_lin_0.001';
# PCA_par['fn_DB'] = 'DB_case0_bi_lin_0.001';
PCA_par['fn_DB'] = 'DB_case1_bi_lin_0.001'
PCA_par['fn_DB'] = PCA_par['fn_DB'] + file_format
 

In [4]:
# Siginifiqance Level for Novelty Detection: Threshold
PCA_par['alpha'] = 0.95; # You can change it, but i recommend the values of 95% or 99%

#################### Do not chane the following options #################### 

# Scaling Method  
PCA_par['SD_type'] = 'Z-score' # Do not change it

# Method for selecting number of the retained PCs
PCA_par['nPC_select_algorithm'] = 'eigengap' # Do not change it

# Delayed number for updating PCs to consider disturbance or measurement error
# PCA_par.n_stall=1;
# PCA_par.n_stall=3;
PCA_par['n_stall'] = 10;

In [5]:
if file_format == '.mat':
    op = spio.loadmat(PCA_par['fn_DB'])
    op = op['op']
elif file_format == '.h5':

    with open(PCA_par['fn_DB'], 'rb') as f:  # Python 3: open(..., 'rb')
        op_ = pickle.load(f)

    op = op_.copy()

In [6]:
PCA_par['x'] = np.zeros((2, 2), dtype = np.int16)
PCA_par['x'][0] =[op['IND_x0'][0], op['IND_x0'][-1]]; # INITIAL TRAINIG DATA
PCA_par['x'][1]=[op['IND_x1'][0], op['IND_x1'][-1]]; # TEST DATA
PCA_par['d_indx'] = op['infor']['D_point'] # Sample index at damage ( 0: no damage // 700: damage at # sample index of 700)

In [7]:
t = op['t']; # Assign Temp. to new variable of t
meas = np.hstack((np.transpose(t), op['f'][:,0:2])) # Assign Measurement to new variable of f

## PLOT RAW DATA

# To be updated 
        

In [8]:
#### MAIN LOOP FOR APCA based on Variable Moving Window PCA ####

### DEFINE HYPER-PARAMETERS OF VMWPCA ###

SD_type = PCA_par['SD_type'] # Scaling Method  
alpha = PCA_par['alpha'] # Siginifiqance Level for Novelty Detection: Threshold
nPC_select_algorithm = PCA_par['nPC_select_algorithm'] # Method for selecting number of the retained PCs

x0 = np.arange(PCA_par['x'][0, 0], PCA_par['x'][0, 1]+1) # Initial training samples
x1 = np.arange(PCA_par['x'][1, 0], PCA_par['x'][1, 1]+1) # Test samples

PC = {}
PC['k_cluster'] = np.arange(0,4)  # of clusters for block-wise linearization
PC['n_stall'] = PCA_par['n_stall'] # Delayed number for updating PCs to consider disturbance or measurement error


In [12]:
## STEP #1: Initial PCs based on initial training data

# Training data set
X0 = meas[x0,:]
numobs, numvars = X0.shape
mu0, sds0 = np.mean(X0, 0), np.std(X0, 0)

# Standardization: Z-score (zero-mean and unit variance)

X0std=Standardization_Data(X0,mu0,sds0,numobs,SD_type); 

# PERFORM PCA

pc = PCA(X0std, standardize=False)
loadings = pc.Wt.T
cov = loadings.dot(np.diag(pc.s)).dot(pc.Wt) / (loadings.shape[0] - 1)
scores = pc.Y
variances = np.sort(np.linalg.eig(cov)[0])[::-1]
tscores = hotelling_tsquared(pc)

# Extracting the retained PC  => Eq. (9) in Ref. (SHM, 2018)
Y = np.max(np.abs(np.diff(variances)))
IX = np.argmax(np.abs(np.diff(variances)))
n_pc = IX




In [13]:
residuals = (X0std - scores[:,0:n_pc+1]*loadings[:,0:n_pc+1].T)
Qdist = np.sqrt(np.sum(np.square(residuals),1));
m_Q=np.mean(Qdist)
V_Q=np.var(Qdist)
V=2*(np.square(m_Q))/V_Q;
distcrit = V_Q/(2*m_Q)*chi2.ppf(alpha,V) #  Threshold limit based on significance level


# Save results of the intial PC model from initial training data

X1 = {}
X1['f0']=X0
X1['mu0']=np.matlib.repmat(mu0,Qdist.shape[0],1)
X1['sds0']=np.matlib.repmat(sds0,Qdist.shape[0],1);

PC['loadings']=loadings 
PC['scores']=scores
PC['n_pc']=n_pc
PC['variances']=variances
PC['tscores']=tscores
PC['update']=1

Q = {}
Q['Qdist']=Qdist
Q['m_Q']=m_Q
Q['V_Q']=V_Q
Q['distcrit']=np.matlib.repmat(distcrit,Qdist.shape[0],1);

In [ ]:
## STEP #2: Post-processing of the initial PC model

In [19]:
Q_lim=Q['distcrit']
PC['fault'] = []; # Pre-allocate faulty sample
PC['Updated'] = np.ones((Q['Qdist'].shape[0],1)) # Sample index of updating PC
# Set minimum and maximum length of moving window size

#  the minimum and maximum length of moving window can be selected roughly based
# 
L_min = 40 # The value of minimum length can be adjusted based on your process
L_max = 1500 # The value of maximum length can be adjusted based on your process

X1['L_min'] = L_min
X1['L_max'] = L_max 
X1['L']=[] 
X1['N_cluster']=[]
X1['BIC']=[]

if Qdist.shape[0] < X1['L_max']:
    X1['L'] = np.matlib.repmat(Qdist.shape[0], Qdist.shape[0],1)
    X1['L0'] = X1['L']
else :
    X1['L'] = np.matlib.repmat(X1['L_max'], Qdist.shape[0],1)
    X1['L0'] = X1['L']

X1['t'] = op['t']


In [23]:
## STEP #3: Update PC models for test samples
## You need to modify this part properly for your application
# Add the last threhold limit (herein, inititla training samples) from the previous PC model

Q['distcrit'] = np.vstack((Q['distcrit'], Q['distcrit'][-1]))

for i in range(0, x1.shape[0]):
    if 1 : 

(201, 1)

In [45]:
X1['f'] = np.vstack((X1['f0'][-X1['L'][-1, 0]:] , meas[x1[i],:]))

array([[21.5       ,  2.84404796, 11.37720822],
       [20.3       ,  2.84394933, 11.39199257],
       [29.        ,  2.8451878 , 11.36806368],
       [26.8       ,  2.84165711, 11.39461585],
       [23.3       ,  2.84681396, 11.38748285],
       [20.9       ,  2.85044361, 11.41267016],
       [31.4       ,  2.83925065, 11.36667678],
       [27.2       ,  2.8404259 , 11.3781241 ],
       [23.4       ,  2.84220226, 11.39214777],
       [21.4       ,  2.84837396, 11.40166918],
       [28.9       ,  2.8387874 , 11.35550245],
       [28.6       ,  2.84059031, 11.3620764 ],
       [21.9       ,  2.84486348, 11.38219784],
       [21.4       ,  2.85021464, 11.37955801],
       [28.9       ,  2.8445952 , 11.34840904],
       [30.3       ,  2.83850917, 11.37642917],
       [23.5       ,  2.84873709, 11.37652655],
       [21.3       ,  2.84736821, 11.39487654],
       [26.5       ,  2.8416803 , 11.37004444],
       [25.9       ,  2.84219521, 11.378228  ],
       [23.3       ,  2.84430016, 11.392

In [42]:
i = 0

In [ ]:

Q.distcrit=[Q.distcrit; Q.distcrit(end,:)];

for i=1:size(x1,2)
    if 1
        =[; ];        
        [X1,PC,Q]=VMWPCA_update(X1,PC,Q,alpha,SD_type,x1(i),i);
    else
        X1.f=[X1.f; meas(x1(i),:)];        
        [X1,PC,Q]=MWPCA_update(X1,PC,Q,alpha,SD_type,x1(i),i);
    end
end

In [46]:
def VMWPCA_update(X0,PC,Q,alpha,SD_type,ind_x,ind): 
    f, mu0, sds0 = X0['f'], X0['mu0'], X0['sds0']
    loadings, n_pc, variances = PC['loadings'], PC['n_pc'], PC['variances'];
    Qdist , distcrit = Q['Qdist'], Q['distcrit'];
    
    ## Standardization (zero-mean)
    numobs1 = 1
    numvars = f.shape[1]
    X1std = Standardization_Data(f[-1,:], mu0[-1,:], sds0[-1,:], numobs1, SD_type);
    
    ## PC_PROJECTION
    scores1 = X1std*loadings[:,0:n_pc[-1,:]];
    residuals_val = X1std - scores1[:,0:n_pc[-1,:]]*loadings[:,0:n_pc[-1,:]].T
    Qdist_val = np.sqrt(np.divide(np.sum(np.square(residuals_val),2), (numvars-2)));
    Qdist = np.vstack((Qdist, Qdist_val))